In [ ]:
!pip install -r requirements.txt

# Inference with ngraph

In [ ]:
from ngraph_onnx.onnx_importer.importer import import_onnx_file
import ngraph as ng
# Import the ONNX file
model = import_onnx_file('model/model.onnx')

# Create an nGraph runtime environment
runtime = ng.runtime(backend_name='CPU')

# Select the first model and compile it to a callable function
emotion_cnn = runtime.computation(model)

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
# Reading image and reshaping
image = Image.open("images/smile.jpg").convert('L')
resized = image.resize((64, 64))

values = np.array(resized).reshape(1, 1,  64, 64)

In [ ]:
# Adding softmax to compute probability of outputs
def _softmax( x):
    e_x = np.exp(x -np.max(x))
    return  e_x/np.sum(e_x)

# compute probabilites and sort
def _postprocess( x):
    prob = _softmax(x)
    prob = np.squeeze(prob)
    classes = np.argsort(prob)[::-1]
    return classes, prob

In [ ]:
classes, prob = _postprocess(emotion_cnn(values))

In [ ]:
# Class mapping
emotion_table = {
    "0": "neutral",
    "1": "happiness",
    "2": 'surprise',
    "3": "sadness",
    "4": "anger",
    "5": "disgust",
    "6": "fear",
    "7": "contempt"
}

In [ ]:
{ emotion_table[str(c)]: str(prob[c]) for c in classes}

In [ ]:
np.sum(prob)

# Seldon Core Microservice

In [2]:
# https://docs.docker.com/compose/install/
!docker-compose up -d 

Starting seldon-core-onnx_emotion_service_1 ... 
ting seldon-core-onnx_emotion_service_1 ... done

In [4]:
# pip install Pillow
from PIL import Image
# pip install numpy
import numpy as np
# pip install requests
import requests
path_to_image = "images/smile.jpg"
image = Image.open(path_to_image).convert('L')
resized = image.resize((64, 64))
values = np.array(resized).reshape(1, 1, 64, 64)
req = requests.post("http://localhost:5000/predict",
              json={"data":{"ndarray": values.tolist()}})
print(req.json())

{'jsonData': {'anger': '0.00013194197', 'contempt': '0.0011185447', 'disgust': '4.5716053e-05', 'fear': '2.5447316e-05', 'happiness': '0.90062636', 'neutral': '0.096829824', 'sadness': '0.0007716461', 'surprise': '0.00045050686'}, 'meta': {}}
